In [ ]:
require 'parser/current'
require 'rubocop-ast'
require 'rubocop'
# opt-in to most recent AST format:
Parser::Builders::Default.emit_lambda              = true

In [ ]:
RuboCop::Version::STRING

# My rules

In [ ]:
NAMESPACE_REPLACEMENTS = {
  :Airflow => "airflow",
  :EnergyPlus => "energyplus",
  :EPJSON => "epjson",
  :GbXML => "gbxml",
  :Gltf => "gltf",
  :ISOModel => "isomodel",
  :Measure => "measure",
  :Model => "model",
  :OSVersion => "osversion",
  :Radiance => "radiance",
  :SDD => "sdd",
}

In [ ]:
class Rule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def initialize(rewriter)
    @rewriter = rewriter
  end

  def create_range(begin_pos, end_pos)
    Parser::Source::Range.new(@rewriter.source_buffer, begin_pos, end_pos)
  end
end


class ReplaceRequire < Rule
  def on_send(node)
    if node.method_name == :require
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      @rewriter.replace(node.loc.expression, "import #{req.value}")
    elsif node.method_name == :require_relative
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      if req.value == "lib/baseline_model"
        @rewriter.replace(node.loc.expression, "from lib.baseline_model import BaselineModel")
      else
        # raise if req.value != "lib/baseline_model"
        puts "Unknown #{req.value}"
      end
    end
  end
end

class ArrayFirst < Rule
  def on_send(node)
    return unless node.method_name == :first

    range = create_range(node.loc.dot.begin_pos, node.loc.expression.end_pos)
    @rewriter.replace(range, '[0]')
  end
end

class AddParenthesesToMethods < Rule
  def on_send(node)
    if node.arguments.empty? && !(node.loc.expression.source_line.end_with?('()'))
      range = create_range(node.loc.expression.end_pos, node.loc.expression.end_pos)
      @rewriter.replace(range, '()')
      # puts node.receiver
    end
  end
end

class ProcessHash < Rule
  def on_hash(node)
    # @rewriter.replace(node.loc.expression, node.loc.expression.source.gsub('=>',':'))
    
    args = []
    node.child_nodes.each do |pair_node|
      raise if !pair_node.pair_type?
      raise if pair_node.child_nodes.size != 2
      raise if !pair_node.child_nodes[0].str_type?
      args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
    end
    @rewriter.replace(node.loc.expression, args.join(', '))
  end
end

class SyntaxBreakingReplacements < Rule
  
  def on_begin(node)
    return
  end
  
  def on_const(node)
    replace_const_namespaces(node)
  end
  
  def on_true(node)
    @rewriter.replace(node.loc.expression, "True")
  end
  
  def on_false(node)
    @rewriter.replace(node.loc.expression, "False")
  end
  
  def on_block(node)
    replace_sort_bys(node)
    replace_each(node)
    replace_each_with_index(node)
  end
  
  def on_if(node)
    
    # DISABLED, it breaks if you have more than two ifs
    # return
    
    return if node.nil?
    return if node.unless? || node.ternary? # || node.modifier_form?
    
    @rewriter.insert_after(node.condition.loc.expression, ':')
    if node.loc.end
      # Remove last end
      @rewriter.replace(node.loc.end, '')
    end
    if node.else_branch && !node.else_branch.if_type? && node.else?
      @rewriter.insert_after(node.else?, ':')
    end
    if node.keyword == 'elsif' 
      @rewriter.replace(node.loc.keyword, 'elif')
    end
  end
  
  def on_send(node)
    replace_puts(node)
    replace_new(node)
    replace_append(node)
    process_raise_if(node)
    replace_select(node)
    replace_each_block_pass(node)
    replace_not_operator(node)
  end
  
  def on_next(node)
    if node.parent.modifier_form?
      rewritten = "#{node.parent.keyword} #{node.parent.condition.source}: continue"
      @rewriter.replace(node.parent.loc.expression, rewritten)
    end
  end
  
  def process_raise_if(node)
    return unless node.send_type?
    return unless node.method_name == :raise
    return unless node.parent.modifier_form?
    
    args = node.arguments? ? "(#{node.arguments.map(&:source).join(', ')})" : ''
  
    rewritten = "#{node.parent.keyword} #{node.parent.condition.source}: raise ValueError#{args}"
    @rewriter.replace(node.parent.loc.expression, rewritten)
  end

  def replace_sort_bys(node)      
    # Definitely breaking the Ruby syntax
    return unless node.method_name == :sort_by

    receiver = node.receiver
    body = node.body
    
    return unless receiver && body

    body_str = node.body.source
    args_str = node.arguments.child_nodes.map{|a| a.source}.join(', ')

    # Build the corrected code
    corrected_code =  "sorted(#{receiver.source}, key=lambda #{args_str}: #{body_str})"

    # Replace the original code with the corrected one
    range = create_range(node.loc.expression.begin_pos, node.loc.expression.end_pos)
    @rewriter.replace(range, corrected_code)
  end
  
  def replace_each_with_index(node)
    return unless node.method_name == :each_with_index
    corrected_code = "for #{node.arguments.reverse.map(&:source).join(', ')} in enumerate(#{node.receiver.source}):"
    range = create_range(node.loc.expression.begin_pos, node.arguments.loc.expression.end_pos)
    @rewriter.replace(range, corrected_code)
    @rewriter.replace(node.loc.end, "") # Remove end
  end
  
  def replace_each(node)
    return unless node.method_name == :each
    corrected_code = "for #{node.arguments.map(&:source).join(', ')} in #{node.receiver.source}:"
    range = create_range(node.loc.expression.begin_pos, node.arguments.loc.expression.end_pos)
    @rewriter.replace(range, corrected_code)
    @rewriter.replace(node.loc.end, "") # Remove end
  end

  def replace_puts(node)
    return unless node.method_name == :puts

    print_content = node.arguments.map(&:source).join(', ')
    f_string = print_content.include?('#{') ? "f" : ''
    print_content.gsub!('#{', '{')

    corrected_code = "print(#{f_string}#{print_content})"

    @rewriter.replace(node.loc.expression, corrected_code)
  end
    
  
  def replace_new(node)
    return unless node.method_name == :new
    range = create_range(node.loc.dot.begin_pos, node.loc.selector.end_pos)
    @rewriter.replace(range, '')
  end
  
  def replace_append(node)
    return unless node.method_name == :<< && node.receiver&.lvar_type?
    corrected_code = "#{node.receiver.source}.append(#{node.arguments.map(&:source).join(', ')})"
    @rewriter.replace(node.loc.expression, corrected_code)
  end
  
  def replace_each_block_pass(node)
    return unless node.method_name == :each || node.method_name == :map
    return if node.block_type?
    return unless !node.block_node 
    return unless node.arguments.one? && node.first_argument.block_pass_type? 
    block_pass = node.first_argument
    return unless block_pass.children.first&.sym_type?
    @rewriter.replace(node.loc.expression, "[x.#{block_pass.children.first.value.to_s}() for x in #{node.receiver.source}]")
  end
  
  def replace_select(node)
    return unless node.method_name == :select
    return if node.block_type?
    return unless node.block_node
    args = node.block_node.arguments.map(&:source).join(', ')
    list_comprehension = "[#{args} for #{args} in  #{node.receiver.source} if #{node.block_node.body.source}]"
    @rewriter.replace(node.loc.expression, list_comprehension)
  end
  
  def replace_not_operator(node)
    return unless node.method_name == :!
    @rewriter.replace(node.loc.selector, 'not ')
  end
  
  def replace_const_namespaces(node)
    # This is **that** breaking but still...
    if !node.parent&.const_type?
      return unless node.namespace&.short_name == :OpenStudio || node.namespace&.namespace&.short_name == :OpenStudio
      @rewriter.replace(node.loc.double_colon, '.')
      return
    end
    #puts "node=#{node}"
    if !node.namespace 
        return unless node.short_name == :OpenStudio
        @rewriter.replace(node.loc.expression, 'openstudio')
        return
    end

    
    puts "node.namespace.short_name=#{node.namespace.short_name}"
    if node.namespace.short_name == :OpenStudio
      raise "#{node.short_name} not found" if !NAMESPACE_REPLACEMENTS.include?(node.short_name)
      # Replace Model with model
      range = create_range(node.loc.double_colon.begin_pos, node.loc.name.end_pos)
      @rewriter.replace(range, "." + NAMESPACE_REPLACEMENTS[node.short_name])
    end
  end
end

class ReplaceCaseWithIf < Rule
  def on_case(node)
    return unless node.conditional?

    condition = node.condition.source
    is_first = true
    when_branches = node.when_branches.map { |when_node| 
        content = process_when_branch(when_node, is_first)
        is_first = false
        content 
    }
    else_branch = process_else_branch(node.else_branch)

    # Build the corrected code
    corrected_code = <<~RUBY
      #{when_branches.join("\n")}
      #{else_branch}
      end
    RUBY

    # Replace the original code with the corrected one
    range = create_range(node.loc.keyword.begin_pos, node.loc.end.end_pos)
    @rewriter.replace(range, corrected_code)
  end

  private

  def process_when_branch(when_node, is_first)
    condition = when_node.conditions.map{ |cond| "#{when_node.parent.condition.source} == #{cond.source}"}.join(' || ')
    body = when_node.body.source
    keyword = is_first ? "if" : "elsif"
    <<~RUBY
      #{keyword} #{condition}
        #{body}
    RUBY
  end

  def process_else_branch(else_branch)
    return '' unless else_branch

    "else\n#{else_branch.source}"
  end
end

# Main

In [ ]:
# Main method.

def replace_gem_version(code)
  return code.gsub('Gem::Version', 'openstudio.VersionString').gsub('OpenStudio.openStudioVersion', 'openstudio.openStudioVersion')
end

def ensure_all_namespaces(code)
  # Some replacements are left out, just force them
  
  code.gsub!('OpenStudio::', 'openstudio.')
  code.gsub!('OpenStudio.', 'openstudio.')
  NAMESPACE_REPLACEMENTS.each do |ruby_sym, replacement|
    code.gsub!(ruby_sym.to_s, replacement)
  end
  code.gsub!('::', '.')
  code.gsub!('.new(', '(')
  code.gsub!('elsif ', 'elif ')
  return code
end

def break_if_continue(code)
  return code.split("\n", -1).map{|line|
    if line.strip.start_with?('if') && line.strip.end_with?(': continue')
      leading_spaces =  line.length - line.lstrip.length
      line.gsub(': continue', ":\n#{' ' * (leading_spaces + 4)}continue")
    elsif (line.strip.start_with?('if') || line.strip.start_with?('elif') || line.strip.start_with?('else')) && line.strip[-1] != ':' 
      line += ':'
    else
      line
    end
  }.join("\n")
end

def replace_not_operator(code)
  return code.gsub('if !', 'if not ')
end

def post_cleanup(code)
  code = replace_gem_version(code)
  code = ensure_all_namespaces(code)
  code = break_if_continue(code)
  code = replace_not_operator(code)
  code = code.gsub('&&', 'and')
  code = code.gsub('||', 'or')
  code = code.gsub('.nil?', 'is None')
  code = code.gsub(' nil', ' None')
  return code
end

def process_file(file)
  return unless File.exist?(file)
  code = File.read(file)  
  return process_code(code)
end

def process_code(
  code, 
  non_breaking_rule_classes=[ReplaceCaseWithIf, ArrayFirst, AddParenthesesToMethods, ReplaceRequire, ProcessHash], 
  breaking_rule_classes=[SyntaxBreakingReplacements]
) 
  code.gsub!('.name.to_s', '.nameString')
  code.gsub!('.name.get', '.nameString')
  
  non_breaking_rule_classes.each do |rule_class|
    puts "Processing #{rule_class}"
    code = process_rule(rule_class, code)
  end  
  code = align_4_spaces(code)

  frozen_lit = "# frozen_string_literal: true\n\n"
  if code.start_with?(frozen_lit)
    code = code[frozen_lit.size..]
  end
  
  breaking_rule_classes.each do |rule_class|
    puts "Processing #{rule_class}"
    code = process_rule(rule_class, code)
  end
  code = post_cleanup(code)
  code
end

def process_rule(rule_class, code)
  source = RuboCop::AST::ProcessedSource.new(code, 2.7)
  source_buffer = source.buffer
  rewriter = Parser::Source::TreeRewriter.new(source_buffer)
  rule = rule_class.new(rewriter)
  source.ast.each_node { |n| rule.process(n) }
  rewriter.process
end


def reformat_python_file(file_name)
  return system("black -l 120 #{file_name}")
end

def port_file(file_name, verbose: false)
  code = process_file(file_name)
  
  #code.gsub!('OpenStudio::Model::', 'openstudio.model.')
  code.gsub!('Dir.pwd()', 'None')
  if verbose
    puts "=" * 80
    puts code
  end

  target_file = File.join(File.dirname(file_name), File.basename(file_name, '.rb') + '.py')
  puts "target file: #{target_file}"

  File.write(target_file, code)
  valid_syntax = reformat_python_file(target_file)
    
  return target_file, valid_syntax
end

In [ ]:
system("echo 'lol'")

In [ ]:
code = %q(
m.getCoilCoolingWaters()[0].setName('VAV Central ChW Coil')

for coil in m.getCoilHeatingWaters():
  if !coil.airLoopHVAC().is_initialized(): continue

  coil.setName('VAV Central HW Coil')


# Rename nodes
for p in m.getPlantLoops():
  prefix = p.nameString()

  for c in p.supplyComponents().reverse():
    if c.to_Node().is_initialized(): continue

    if c.to_ConnectorMixer().is_initialized():
      c.setName("#{prefix} Supply ConnectorMixer")
    elsif c.to_ConnectorSplitter().is_initialized()
      c.setName("#{prefix} Supply ConnectorSplitter")
    else
)

In [ ]:
new_lines = code.lines.each do |line|
  if line.strip.s
  end
end

In [ ]:
line = "    if c.to_Node().is_initialized(): continue\n"

In [ ]:
line.strip.start_with?('if') && line.strip.end_with?(': continue')

In [ ]:
leading_spaces =  line.length - line.lstrip.length

In [ ]:
puts line.gsub(': continue', ":\n#{' ' * (leading_spaces + 4)}continue")

## I HAVE AN ORDER ISSUE: MY RULES BREAK THE RUBY SYNTAX!

## Test: reindent

In [ ]:
code = %q(
10.times do |i|
if i == 0 || i == 1
           puts "zero"
    elsif i == 4
puts "4"
else
  puts "else"
  if i < 5 or i > 10
    puts "wrong"
end
  end
end
)

In [ ]:
options, paths = RuboCop::Options.new.parse(['-a', '--only', 'Layout/IndentationWidth,Layout/EndAlignment,Layout/ElseAlignment,Layout/IndentationConsistency',
  '--stdin', code])
paths

In [ ]:
def align_4_spaces(code)
  config_path = 'temp_config.yml'
  File.write(config_path, '''Layout/IndentationWidth:
    Width: 4
  ''')
  File.write('temp.rb', code)
  options, paths = RuboCop::Options.new.parse(['-a', '--format=simple', '--only', 'Layout/IndentationWidth,Layout/EndAlignment,Layout/ElseAlignment,Layout/IndentationConsistency',
    'temp.rb'
  ])
  store = RuboCop::ConfigStore.new
  store.options_config = config_path
  runner = RuboCop::Runner.new(options, store)
  runner.run(paths)
  reformatted_code = File.read('temp.rb')
  File.delete('temp.rb')
  File.delete('temp_config.yml')
  return reformatted_code
end

In [ ]:
puts align_4_spaces(code)

## Test: replace puts

In [ ]:
code = %q(
systems = [1, 2, 3]
systems.each_with_index do |system, i|
    puts "#{i}, #{system}"
    puts "caca"
end
)

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

## Test: replace namespaces

In [ ]:
code = %q(m = OpenStudio::Model::Model.new()
a = OpenStudio::Model::AvailabilityManagerNightVentilation.new(model)
t = OpenStudio::Time.new(0, 1, 0, 0)
p = OpenStudio::Point3d.new(1, 1 ,1)
)

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

## Test: array append

In [ ]:
code = %q(a = []
a << [1, 2]
a << 2
2 << 8
)
source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

## Test next if, raise if

In [ ]:
class IfReplacements < Rule

  def on_if(node)
    
    # DISABLED, it breaks if you have more than two ifs
    # return
    
    return if node.nil?
    return if node.unless? || node.ternary? || node.modifier_form?
    
    puts node
    @rewriter.insert_after(node.condition.loc.expression, ':')
    if node.loc.end
      # Remove last end
      @rewriter.replace(node.loc.end, '')
    end
    if node.else_branch && !node.else_branch.if_type? && node.else?
      @rewriter.insert_after(node.else?, ':')
    end
    if node.keyword == 'elsif' 
      @rewriter.replace(node.loc.keyword, 'elif')
    end
  end
  
  def on_next(node)
    if node.parent.modifier_form?
      rewritten = "#{node.parent.keyword} #{node.parent.condition.source}: continue"
      @rewriter.replace(node.parent.loc.expression, rewritten)
    end
  end
  
  def on_send(node)
    process_raise_if(node)
  end
  
  def process_raise_if(node)
    return unless node.send_type?
    return unless node.method_name == :raise
    return unless node.parent.modifier_form?
    
    args = node.arguments? ? "(#{node.arguments.map(&:source).join(', ')})" : ''
  
    rewritten = "#{node.parent.keyword} #{node.parent.condition.source}: raise ValueError#{args}"
    @rewriter.replace(node.parent.loc.expression, rewritten)
  end
end

In [ ]:
code = %q(10.times do |i|
  next if i == 4
  raise if i == 10
  raise 'message' if i == 10
end
)
source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = IfReplacements.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.next_type?
  node
}[0]

In [ ]:
node.parent.ternary?

In [ ]:
node.source

In [ ]:
"#{node.parent.keyword} #{node.parent.condition.source}: continue"

In [ ]:
source.ast.each_node

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.send_type?
  node
}[5]

In [ ]:
node.parent.source

In [ ]:
node.arguments.map(&:source)

## Test if :  Remove end, add semi colons

In [ ]:
class IfReplacements < Rule
  
  def on_if(node)
    
    # DISABLED, it breaks if you have more than two ifs
    # return
    
    return if node.nil?
    return if node.unless? || node.ternary?
    
    puts node
    @rewriter.insert_after(node.condition.loc.expression, ':')
    if node.loc.end
      # Remove last end
      @rewriter.replace(node.loc.end, '')
    end
    if node.else_branch && !node.else_branch.if_type? && node.else?
      @rewriter.insert_after(node.else?, ':')
    end
    if node.keyword == 'elsif' 
      @rewriter.replace(node.loc.keyword, 'elif')
    end
  end
end

In [ ]:
code = %q(
if i == 4
    puts "yes"
elsif i == 2
    puts "also"
else
    puts "no"
end
)

code = %q(if i == 2
  puts "yes"
end

if i == 4
  puts "no"
end
)

code = %q(
model.getPlantLoops.each do |p|
  prefix = p.name.to_s

  p.supplyComponents.reverse.each do |c|
    next if c.to_Node.is_initialized

    if c.to_ConnectorMixer.is_initialized
      c.setName("#{prefix} Supply ConnectorMixer")
    elsif c.to_ConnectorSplitter.is_initialized
      c.setName("#{prefix} Supply ConnectorSplitter")
    else

      obj_type = c.iddObjectType.valueName
      obj_type_name = obj_type.gsub('OS_', '').gsub('_', '')

      if c.to_PumpVariableSpeed.is_initialized
        c.setName("#{prefix} VSD Pump")
      elsif c.to_PumpConstantSpeed.is_initialized
        c.setName("#{prefix} CstSpeed Pump")
      elsif c.to_HeaderedPumpsVariableSpeed.is_initialized
        c.setName("#{prefix} Headered VSD Pump")
      elsif c.to_HeaderedPumpsConstantSpeed.is_initialized
        c.setName("#{prefix} Headered CstSpeed Pump")
      elsif !c.to_CentralHeatPumpSystem.is_initialized
        c.setName("#{prefix} #{obj_type_name}")
      end
    end
  end
end
)

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
#source = RuboCop::AST::ProcessedSource.from_file('model/simulationtests/additional_props.rb', 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

In [ ]:
source = RuboCop::AST::ProcessedSource.new(code, 2.7)
#source = RuboCop::AST::ProcessedSource.from_file('model/simulationtests/additional_props.rb', 2.7)
source_buffer = source.buffer
@rewriter = Parser::Source::TreeRewriter.new(source_buffer)

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.if_type?
  node
}[0]

In [ ]:
node.condition.loc.expression

In [ ]:
node.condition.loc

In [ ]:
@rewriter.insert_after(node.condition.loc.expression, ':')
if node.loc.end
  @rewriter.replace(node.loc.end, '')
end
if node.else_branch && !node.else_branch.if_type? && node.else?
  @rewriter.insert_after(node.else?, ':')
end
if node.keyword == 'elsif' 
  @rewriter.replace(node.loc.keyword, 'elif')
end
puts rewriter.process

In [ ]:
source.ast.each_node {|n|
  next unless n.if_type?
  
   # Remove last end:
  @rewriter.insert_after(node.condition.loc.expression, ':')
  if node.loc.end
    @rewriter.replace(node.loc.end, '')
  end
  if !node.else_branch.if_type? && node.else?
    @rewriter.insert_after(node.else?, ':')
  end
  if node.keyword == 'elsif' 
    @rewriter.replace(node.loc.keyword, 'elif')
  end
};

In [ ]:
  def on_if(node)
    
    # DISABLED, it breaks if you have more than two ifs
    # return
    
    return if node.nil?
    return if node.unless? || node.ternary?
    
    # Remove last end:
    @rewriter.insert_after(node.condition.loc.expression, ':')
    if node.loc.end
      @rewriter.replace(node.loc.end, '')
    end
    if !node.else_branch.if_type? && node.else?
      @rewriter.insert_after(node.else?, ':')
    end
    if node.keyword == 'elsif' 
      @rewriter.replace(node.loc.keyword, 'elif')
    end
  end

In [ ]:
require 'parser/current'

class RemoveDo < Parser::TreeRewriter
  def on_while(node)
    # Check if the statement starts with "do"
    if node.location.begin.is?('do')
      remove(node.location.begin)
    end
  end
  
  def on_if(node)
    
    # DISABLED, it breaks if you have more than two ifs
    # return
    
    return if node.nil?
    #return if node.unless? || node.ternary?
    
    # Remove last end:
    @rewriter.insert_after(node.condition.loc.expression, ':')
    if node.loc.end
      @rewriter.remove(node.loc.end)
    end
    if !node.else_branch.if_type? && node.else?
      @rewriter.insert_after(node.else?, ':')
    end
    if node.keyword == 'elsif' 
      @rewriter.replace(node.loc.keyword, 'elif')
    end
  end
end

code = <<-EOF
while true do
  puts 'hello'
end
if i == 2
  puts "yes"
end

if i == 4
  puts "no"
end
EOF

ast           = Parser::CurrentRuby.parse code
buffer        = Parser::Source::Buffer.new('(example)', source: code)
rewriter      = RemoveDo.new

# Rewrite the AST, returns a String with the new form.
puts rewriter.rewrite(buffer, ast)


In [ ]:
ast.location

In [ ]:
node = source.ast.each_node.to_a[2]
node.if_type?

In [ ]:
rule.process(source.ast.each_node.to_a[1])

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.if_type?
  node
}[0]

In [ ]:
node.type

In [ ]:
node.keyword == 'elsif' 
node.loc.keyword, 'elif'

In [ ]:
# Remove last end:
rewriter.insert_after(node.condition.loc.expression, ':')
rewriter.replace(node.loc.end, '')
rewriter.process

In [ ]:
!node.else_branch.if_type? && node.else?

In [ ]:
node.loc

In [ ]:
node.loc.end

## Test replace Gem::Version

In [ ]:
code = %q(
if Gem::Version(OpenStudio.openStudioVersion()) > Gem::Version('2.7.1')
)

In [ ]:
puts replace_gem_version(code)

## Test replace true / false

In [ ]:
code = %q(
x = true
x = false
)

In [ ]:
class TrueFalseRule < Rule
  def on_true(node)
    @rewriter.replace(node.loc.expression, "True")
  end
  def on_false(node)
    @rewriter.replace(node.loc.expression, "False")
  end

end

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = TrueFalseRule.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
rewriter.process

## Test replace array.first

In [ ]:
code = %q(
x = [1, 2]
a = x.first
)

In [ ]:
class ArrayFirst < Rule
  def on_send(node)
    replace_array_first(node)
  end

  def replace_array_first(node)
    return unless node.method_name == :first

    range = create_range(node.loc.dot.begin_pos, node.loc.expression.end_pos)
    @rewriter.replace(range, '[0]')
  end
end

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = ArrayFirst.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

## Test select and each/map block_pass

In [ ]:
code = %q(
surfaces = model.getSurfaces.select { |s| s.outsideBoundaryCondition == 'Outdoors' }.sort_by { |s| s.name.to_s }

model.getSurfaces().select() { |s| s.outsideBoundaryCondition() == 'Outdoors' }

m.getPipeAdiabatics().each(&:remove)

x = m.getPipeAdiabatics().each(&:nameString)
)

In [ ]:
class SelectRule < Rule
  def on_send(node)
    replace_select(node)
    replace_each_block_pass(node)
  end
  
  def replace_each_block_pass(node)
    return unless node.method_name == :each || node.method_name == :map
    return if node.block_type?
    return unless !node.block_node 
    return unless node.arguments.one? && node.first_argument.block_pass_type? 
    block_pass = node.first_argument
    return unless block_pass.children.first&.sym_type?
    @rewriter.replace(node.loc.expression, "[x.#{block_pass.children.first.value.to_s}() for x in #{node.receiver.source}]")
  end
  
  def replace_select(node)
    return unless node.method_name == :select
    return if node.block_type?
    return unless node.block_node
    args = node.block_node.arguments.map(&:source).join(', ')
    list_comprehension = "[#{args} for #{args} in  #{node.receiver.source} if #{node.block_node.body.source}]"
    @rewriter.replace(node.loc.expression, list_comprehension)
  end
end

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
#rule = SelectRule.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

## Test ! (not) operator

In [ ]:
code = %q(
x = true
if !x
  puts "yes"
end

next if !x.airLoopHVAC().is_initialized()
)

In [ ]:
class NotOperator < Rule
  def on_send(node)
    replace_not_operator(node)
  end
  
  def replace_not_operator(node)
    return unless node.method_name == :!
    @rewriter.replace(node.loc.selector, 'not ')
  end
end

source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = SyntaxBreakingReplacements.new(rewriter)
#rule = NotOperator.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

In [ ]:
puts process_code(code)

In [ ]:
node = source.ast.each_node.select{|n|
  next if !n.send_type?
  next if n.method_name != :!
  n
}[1]

In [ ]:
node.receiver.source

# Test on files

In [ ]:
file_name = 'model/simulationtests/humidity_control.rb'
file_name = 'model/simulationtests/test_port.rb'
file_name = 'model/simulationtests/baseline_sys01.rb'
file_name = 'model/simulationtests/baseline_sys02.rb'

In [ ]:
# ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"].each do |sys_num|
#   port_file("model/simulationtests/baseline_sys#{sys_num}.rb")
# end

In [ ]:
file_name = 'model/simulationtests/availability_managers.rb'
file_name = 'model/simulationtests/daylighting_devices.rb'
file_name = 'model/simulationtests/dual_duct.rb'
file_name = 'model/simulationtests/foundation_kiva_customblocks.rb' # Example of next if
file_name = 'model/simulationtests/centralheatpumpsystem.rb' # Example of next if
file_name = 'model/simulationtests/output_objects_2.rb' # Example of true / false constants
file_name = 'model/simulationtests/airterminal_inletsidemixer.rb' # Example of Array.first
file_name = 'model/simulationtests/daylighting_devices.rb' # Example of previously left over namespace stuff
file_name = 'model/simulationtests/airterminal_fourpipebeam.rb' # Example of if xxx : continue
port_file(file_name, verbose: false)

In [ ]:
Dir.glob('model/simulationtests/a*.rb').sort

# Port all

In [ ]:
ported_files = []
invalid_syntax_files = []
failed_files = []
Dir.glob('model/simulationtests/*.rb').sort.each do |file|
  begin
    target_file, is_valid_syntax = port_file(file)
    if is_valid_syntax
      ported_files << target_file
    else
      invalid_syntax_files << target_file
    end
  rescue
    puts "Failed on #{file}"
    failed_files << file
  end
end

In [ ]:
failed_files

In [ ]:
ported_files.size

In [ ]:
invalid_syntax_files.size

In [ ]:
`black -l 120 model/simulationtests/additional_props.py`

In [ ]:
reformat_python_file

In [ ]:
invalid_syntax_files.reject!{|f| reformat_python_file(f)}

In [ ]:
invalid_syntax_files.size

In [ ]:
File.basename(ported_files[0]).sub('.py', '_py')

In [ ]:
`openstudio --version`

In [ ]:
code = File.read(file_name)

code.gsub!('.name.to_s', '.nameString')
frozen_lit = "# frozen_string_literal: true\n\n"
if code.start_with?(frozen_lit)
  code = code[frozen_lit.size..]
end
source = RuboCop::AST::ProcessedSource.new(code, 2.7)
source_buffer = source.buffer
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = ReplaceCaseWithIf.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

# Test bed

In [ ]:
code = "require openstudio; model = OpenStudio::Model::Model.new\n"
source = RuboCop::AST::ProcessedSource.new(code, 2.7)

In [ ]:
class MyRule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def on_send(node)
    # puts "I found a on_send! #{node}"
    receiver_node, method_name, *arg_nodes = *node

    receiver_node = process(receiver_node) if receiver_node
    node.updated(nil, [
      receiver_node, method_name, *process_all(arg_nodes)
    ])
    if method_name == :require
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      puts req.value
    elsif method_name == :require_relative
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      raise if req.value != "lib/baseline_model"
      puts req.value
    end
  end
end

rule = MyRule.new
source.ast.each_node { |n| rule.process(n) }

In [ ]:
file_name = "model/simulationtests/absorption_chillers.rb"
file_name = "model/simulationtests/availability_managers.rb"

file_name = "model/simulationtests/afn_single_zone_ac.rb"
file_name = "model/simulationtests/air_terminals.rb"

In [ ]:
source_code = File.read(file_name)
source = RuboCop::AST::ProcessedSource.from_file(file_name, 2.7);
source_buffer = source.buffer

In [ ]:
rewriter = Parser::Source::TreeRewriter.new(source_buffer)
rule = ReplaceCaseWithIf.new(rewriter)
source.ast.each_node { |n| rule.process(n) }
puts rewriter.process

In [ ]:
# code = "dehumidify_sch = OpenStudio::Model::ScheduleConstant.new(model)"
# source = RuboCop::AST::ProcessedSource.new(code, 2.7)

In [ ]:
node = source.ast.each_node.to_a[48]
node

In [ ]:
source.ast.each_node.to_a[68]

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.if_type?
  node
}[0]

In [ ]:
source.ast

In [ ]:
node.parent&.const_type?

In [ ]:
node.short_name

In [ ]:
node.short_name

In [ ]:
node.loc.expression

In [ ]:
node.loc.name.source

In [ ]:
replacements[node.short_name]

In [ ]:
class ReplaceOpenStudioNamespace < Rule
  def on_const(node)
      
    puts "node=#{node}"
    return unless node.parent&.const_type?
    puts "OK, parent is const_type"
    replacements = {
      :Airflow => "airflow",
      :EnergyPlus => "energyplus",
      :EPJSON => "epjson",
      :GbXML => "gbxml",
      :Gltf => "gltf",
      :ISOModel => "isomodel",
      :Measure => "measure",
      :Model => "model",
      :OSVersion => "osversion",
      :Radiance => "radiance",
      :SDD => "sdd",
    }
    
    # Check if the node represents the top-level OpenStudio namespace
    if !node.namespace && node.short_name == :OpenStudio
      # Replace OpenStudio with openstudio
      @rewriter.replace(node.loc.expression, 'openstudio')
    else
      puts "node.namespace.short_name=#{node.namespace.short_name}"
      if node.namespace.short_name == :OpenStudio
        # Replace Model with model
        @rewriter.replace(node.loc.name, replacements[node.short_name])
      end
    end
  end
end

In [ ]:
replacements = {
  :Airflow => "airflow",
  :EnergyPlus => "energyplus",
  :EPJSON => "epjson",
  :GbXML => "gbxml",
  :Gltf => "gltf",
  :ISOModel => "isomodel",
  :Measure => "measure",
  :Model => "model",
  :OSVersion => "osversion",
  :Radiance => "radiance",
  :SDD => "sdd",
}

In [ ]:
node.loc.expression

In [ ]:
node.loc.name

In [ ]:
node.child_nodes(

In [ ]:
node.const_name == :OpenStudio && node.parent && node.parent.const_type?

In [ ]:
node.const_type?

In [ ]:
node.const_name

In [ ]:
node.parent.const_type?

In [ ]:
!node.namespace && node.short_name == :OpenStudio

In [ ]:
!node.namespace && node.short_name == :OpenStudio

In [ ]:
body.

In [ ]:
node.method_name

In [ ]:
node.child_nodes[0].method_name

In [ ]:
node.child_nodes[1]

In [ ]:
node.child_nodes[2]

In [ ]:
$LIB_METHOD_NAMES = ['add_geometry',
 'add_windows',
 'set_constructions',
 'add_daylighting',
 'add_hvac',
 'set_space_types',
 'add_thermostats',
 'add_design_days',
 'force_year_description',
 'save_openstudio_osm',
 'add_standards',
 'add_schedule',
 'add_water_heater',
 'add_swh_loop',
 'add_swh_end_uses',
 'rename_loop_nodes',
 'rename_air_nodes'
]

In [ ]:
print(source_code[node.loc.expression.begin_pos...node.loc.expression.end_pos])
node.loc.expression.source.gsub('=>',':')

In [ ]:
node.send_type?

In [ ]:
node.method_name

In [ ]:
source_code[node.loc.selector.begin_pos...node.loc.selector.end_pos]

In [ ]:
source_code

In [ ]:
node = source.ast.each_node.to_a[8]
node

In [ ]:
node.method_name

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.case_type?
  node
}[0]

In [ ]:
node.loc

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.send_type?
  next if ! $LIB_METHOD_NAMES.include?(node.method_name.to_s)
  node
}[1]

In [ ]:
hash_node = node.arguments[0]
hash_node

In [ ]:
args = []
hash_node.child_nodes.each do |pair_node|
  raise if !pair_node.pair_type?
  raise if pair_node.child_nodes.size != 2
  raise if !pair_node.child_nodes[0].str_type?
  args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
end
puts args.join(', ')

In [ ]:
pair_node = hash_node.child_nodes[0]
pair_node

In [ ]:
pair_node.child_nodes.size

In [ ]:
str_node = pair_node.child_nodes[0]

In [ ]:
str_node.value

In [ ]:
"#{pair_node.child_nodes[0].loc.expression.source}=#{pair_node.child_nodes[1].loc.expression.source}"

In [ ]:
child_node.child_nodes[0].value

In [ ]:
child_node.child_nodes[1].value

In [ ]:
str_node = child_node.child_nodes[0]
str_node

In [ ]:
node.loc.expression.source.gsub('=>',':')

In [ ]:
print(source_code[node.loc.expression.begin_pos...node.loc.expression.end_pos])
node.loc.expression.source.gsub('=>',':')

In [ ]:
node.loc.expression.begin_pos

In [ ]:
node = source.ast.each_node.to_a[24]
node

In [ ]:
node.if?

In [ ]:
n = node.child_nodes[0]
n

In [ ]:
n.method_name

In [ ]:
source_code[113...116]

In [ ]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

In [ ]:
source_code[node.loc.dot.begin_pos...node.loc.expression.end_pos]

In [ ]:
node.loc.expression.end

In [ ]:
source.ast.each_node.to_a[8]

In [ ]:
node.loc

In [ ]:
node.loc.expression.source_line.end_with?'()'

In [ ]:
node.arguments.empty?

In [ ]:
"BaselineModel.new".size

In [ ]:
node.receiver

In [ ]:
source_code[113...116]

In [ ]:
node.child_nodes

In [ ]:
node = source.ast.each_node.to_a[76]
node

In [ ]:
node = source.ast.each_node.to_a[77]
node

In [ ]:
node = source.ast.each_node.to_a[98]

In [ ]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

In [ ]:
node.arguments

In [ ]:
node.method_name

In [ ]:
node.sibling_index

In [ ]:
x = node.receiver
x

In [ ]:
x.const_type?

In [ ]:
x.module_name?

In [ ]:
x.class_name?

In [ ]:
x

In [ ]:
x.const_type?

In [ ]:
x.children[0].children[0]

In [ ]:
x.children[0]

In [ ]:
node.child_nodes

In [ ]:
x.const_name

In [ ]:
receiver_node, method_name, *arg_nodes = *node

In [ ]:
node.method_name

In [ ]:
node.child_nodes

In [ ]:
req = arg_nodes[0]

In [ ]:
req.str_type?

In [ ]:
req.value .

In [ ]:
node.loc.selector

In [ ]:
"require 'openstudio'".size

In [ ]:
"require".size

In [ ]:
node.loc.selector

In [ ]:
node.loc.expression.begin_pos

In [ ]:
node.loc.expression.end_pos

In [ ]:
node.loc